# sábado 8 de julio de 2023

In [55]:
# Manejo de datos 
import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium
from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # permite ignorar los warnings de python 
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # empezar maximizado
opciones.add_argument('user.data-dir=selenium') # guarda las cookies
opciones.add_argument('--incognito') # incognito window

In [56]:
urls = []
flag = 0
flag2 = 0

driver = webdriver.Chrome()

driver.get("https://www.dia.es/") 
sleep(3)

# aceptamos las cookies 
driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
sleep(2)

# clicka productos
driver.find_element("css selector", '#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > div > button').click()
sleep(2)

# para cada una de las categorías principales
# poner un max excesivo
for category in range(1,31):

    while 1:

        # intenta clickar
        try:
            
            driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/a').click()
            flag2 = 0

            for subcategory in range (1,31):

                while 1:

                    try:
                        
                        urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/ul/div[{subcategory}]/a').get_attribute("href"))
                        sleep(0.1)
                        break

                    except:

                        flag2 = 1
                        break
                    
                if flag2:
                    break
            
            sleep(1)
            break
        
        # break si se acaban
        except:

            flag = 1
            break

    if flag:

        break

driver.quit()

In [57]:
# dictio con los resultados del scrapeo
resultados_dia = {'supermarket': [],
                'category': [],
                'subcategory': [],
                'name': [], 
                'price': [], 
                'reference_price': [],
                'reference_unit': [],
                'insert_date': [],
                'product_id': [],
                'url': []}

url = ['https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001', 
       'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206']

# creación del driver de chrome
driver = webdriver.Chrome()

# para cada uno de los urls que han salido del primer Selenium
for u in urls:

        # get
        driver.get(u)
        sleep(5)

        # acepta las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parsea html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # listas resultados primera página
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        id = soup.find_all('div', attrs={'data-test-id': 'product-card'})
        
        # apendea los resultados de las listas al dictio extrayendo texto de labels y retocando con métodos de strings
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for i in id:

                if bool(re.findall('\d{3,6}(?:P\d+)?', str(i))[0]):
                        resultados_dia['product_id'].append(re.findall('\d{3,6}(?:P\d+)?', str(i))[0])
                        resultados_dia['url'].append('https://www.dia.es/'+cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "-")+'/'+sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "-")+'/p/'+re.findall('\d{3,6}(?:P\d+)?', str(i))[0])
        
                else:
                        resultados_dia['product_id'].append(np.nan)
                        resultados_dia['url'].append(np.nan)
                        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€').replace(',', '.'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' (').replace(',', '.'))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):

                        # intenta clickar el botón
                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(2)
                        
                        except:
                                # para cuando hay botones flecha (+5 páginas)
                                try:
                                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div[1]/a[2]').click()
                                        sleep(2)
                                except:
                                        pass

                                pass
                        
                        # SE REPITE EL PROCESO ANTERIOR PARA EL RESTO DE PÁGINAS:
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))

                        for i in id:

                                if bool(re.findall('\d{3,6}(?:P\d+)?', str(i))[0]):
                                        resultados_dia['product_id'].append(re.findall('\d{3,6}(?:P\d+)?', str(i))[0])
                                        resultados_dia['url'].append('https://www.dia.es/'+cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "-")+'/'+sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "-")+'/p/'+re.findall('\d{3,6}(?:P\d+)?', str(i))[0])
                        
                                else:
                                        resultados_dia['product_id'].append(np.nan)
                                        resultados_dia['url'].append(np.nan)

                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€').replace(',', '.'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' (').replace(',', '.'))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        # resetea nº botones
        botones = []

# cierra driver
driver.quit()

# show
resultados_dia

{'supermarket': ['dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
 

In [58]:
df = pd.DataFrame(resultados_dia)

df.to_csv("../dia.csv", index=False, sep= ",")

ValueError: All arrays must be of the same length

In [59]:
len(resultados_dia['product_id'])

5063

In [60]:
len(resultados_dia['name'])

5132

In [61]:
resultados_dia['product_id']

['273810',
 '273957',
 '273750',
 '273749',
 '273737',
 '273965',
 '146710',
 '74042',
 '178002',
 '74043',
 '273960',
 '273961',
 '273736',
 '273733',
 '162311',
 '255685',
 '273747',
 '273748',
 '137167',
 '273954',
 '293647',
 '273959',
 '274755',
 '274754',
 '274104',
 '274101',
 '273964',
 '273962',
 '273958',
 '273956',
 '273955',
 '273953',
 '273952',
 '273756',
 '273751',
 '273746',
 '273744',
 '273742',
 '273740',
 '273738',
 '273732',
 '273145',
 '263497',
 '209382',
 '207576',
 '207575',
 '190136',
 '187756',
 '182582',
 '170157',
 '148433',
 '148432',
 '100807',
 '273442',
 '273443',
 '101581',
 '283361',
 '283362',
 '125955',
 '1062',
 '275707',
 '162106',
 '195181',
 '131234',
 '274067',
 '1306',
 '274068',
 '256940',
 '283363',
 '67436',
 '288206',
 '274051',
 '274059',
 '292098',
 '86666',
 '262594',
 '274055',
 '274056',
 '265993',
 '274060',
 '274058',
 '190103',
 '274061',
 '293068',
 '274057',
 '125958',
 '292099',
 '274053',
 '274138',
 '273976',
 '273988',
 '27414

In [64]:
df = pd.DataFrame(resultados_dia['product_id'])

df.to_csv("../a.csv", index=False, sep= ",")

In [63]:
df

,0
0,273810
1,273957
2,273750
3,273749
4,273737
...,...
5058,115081
5059,114374
5060,292170
5061,288260
